In [2]:
import os
import sys
import nltk
import numpy as np
import string
import xml.etree.ElementTree as ET
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tag import CRFTagger
from kbbi import KBBI

# np.set_printoptions(threshold=sys.maxsize)

def load_xml(path, size, start=0):
    list_xml = []

    # Loop each file in dir path
    for file in os.listdir(path):
        # Get absolute path
        fullname = os.path.join(path, file)
        
        # Parsing XML
        xml = ET.parse(fullname)
        
        # Append XMLTree into list_xml
        list_xml.append(xml)
    
    return list_xml[start:size+start]


def tokenization(list_xml):
    tokens = []

    # Loop each xml in list_xml
    for xml in list_xml:
        # Get text of tag <judul></judul> then remove punctuation
        judul = xml.find('judul').text
        
        # Get text of tag <kata_kunci></kata_kunci> then remove punctuation
        kata_kunci = xml.find('kata_kunci').text
        
        # Get text of tag <isi></isi> then remove punctuation
        isi = xml.find('isi').text
    
        # Append new tokens into current token (new tokens concat 3 tags with spaces)
        tokens = [*tokens, *nltk.word_tokenize(judul + " " + kata_kunci + " " + isi)]
    
    return tokens

def casefolding(tokens):
    # Case folding each token
    tokens = np.char.lower(tokens)
    return tokens


def remove_punctuation(tokens):
    # Punctuation symbols
    symbols = "0123456789!\"'#$%&()*+-.,/:;<=>?@[\]^_`{|}~\n"
    
    # Iterate each punctuation symbol check if token contain punctuation symbol
    for i in range(len(symbols)):
        tokens = np.char.replace(tokens, symbols[i], '')

    # Remove empty whitespace
    tokens = list(filter(None,tokens))
    return tokens

def remove_stopword(tokens):
    # Load stopwords into list_stopword
    list_stopword =  set(stopwords.words('indonesian'))
    new_tokens = []
    
    # Iterate each tokens. if token didn't match with stopword then append to new_tokens list
    for t in tokens:
        if t not in list_stopword:
            new_tokens.append(t)

    return new_tokens

def stemming(tokens):
    # Initialize stemmer
    factory = StemmerFactory()
    # Create stemmer
    stemmer = factory.create_stemmer()
    
    new_tokens = []
    # Iterate each tokens then apply stemming
    for t in tokens:
        new_tokens.append(stemmer.stem(t))
 
    return new_tokens

def pos_tagging(tokens):
    # Create POS Tagger
    ct = CRFTagger()
    
    # Train Post tagger with all_indo_man_tag_corpus_model.crf.tagger data.
    ct.set_model_file('all_indo_man_tag_corpus_model.crf.tagger')
    
    # Do POS Tagging
    result = ct.tag_sents([tokens])
    return result

def count_tagging(list_pos):
    verb = 0
    noun = 0
    adjective = 0
    
    # Count each list
    for pos in list_pos:
        if pos[1] in ["VB", "VBD", "VBG", "VBN", "VBP", "VBZ"]:
            verb += 1
        elif pos[1] in ["NN", "NNS", "NNP", "NNPS"]:
            noun += 1
        elif pos[1] in ["JJ", "JJR", "JJS"]:
            adjective += 1
    
    print(f"Verb:{verb}\nNoun:{noun}\nAdjective:{adjective}")
    
def mistaken_stemming(tokens):
    mistaken_tokens = []
    
    # Loop over tokens
    for token in tokens:
        # Try if token doesn't exist in KBBI
        try:
            result = KBBI(token)
            try:
                # Get base_word of token
                base_word = result.serialisasi()[token][0]['kata_dasar'][0]
                
                # if token doens't match with base_word, append to mistaken_tokens/
                if token != base_word:
                    mistaken_tokens.append(token)
            except IndexError:
                # Skip error if word is base_word
                pass
        except:
            # Append token if doens't exist in KBBI
            mistaken_tokens.append(token)
    return mistaken_tokens

In [5]:
list_xml = load_xml('source', 1)
print(f"Total document:{len(list_xml)}")

Total document:1


In [7]:
tokens = tokenization(list_xml)
print(f"Total token:{len(tokens)}\n")
print(tokens)

Total token:445

['Puji', 'Ekonomi', 'RI', ',', 'IMF', 'Ramal', 'Pertumbuhan', 'Capai', '5,1', 'Persen', 'Ekonomi', ',', 'Keuangan', ',', 'Suku', 'Bunga', 'International', 'Monetary', 'Fund', '(', 'IMF', ')', 'memproyeksi', 'pertumbuhan', 'ekonomi', 'Indonesia', 'hingga', 'akhir', 'tahun', 'nanti', 'hanya', 'akan', 'mencapai', '5', 'persen', '.', 'Namun', ',', 'tahun', 'depan', ',', 'pertumbuhannya', 'berpotensi', 'sedikit', 'menguat', 'hingga', 'tembus', '5,1', 'persen', '.', 'Dewan', 'Eksekutif', 'IMF', 'Luis', 'E', 'Breuer', 'menyebut', 'perekonomian', 'Indonesia', 'terus', 'menunjukkan', 'perbaikan', '.', 'Hal', 'ini', 'didukung', 'oleh', 'kebijakan', 'makroekonomi', 'yang', 'baik', ',', 'serta', 'reformasi', 'struktural', 'yang', 'terus', 'berlanjut', '.', '``', 'Pemerintah', 'dan', 'otoritas', 'secara', 'terampil', 'mampu', 'melewati', 'tantangan', 'ekonomi', 'terkini', 'yang', 'tengah', 'melanda', 'ekonomi', 'global', '.', 'Pertumbuhan', 'tetap', 'kuat', ',', 'inflasi', 'menurun

In [8]:
tokens = remove_punctuation(tokens)
print(f"Total token:{len(tokens)}\n")
print(tokens)

Total token:370

['Puji', 'Ekonomi', 'RI', 'IMF', 'Ramal', 'Pertumbuhan', 'Capai', 'Persen', 'Ekonomi', 'Keuangan', 'Suku', 'Bunga', 'International', 'Monetary', 'Fund', 'IMF', 'memproyeksi', 'pertumbuhan', 'ekonomi', 'Indonesia', 'hingga', 'akhir', 'tahun', 'nanti', 'hanya', 'akan', 'mencapai', 'persen', 'Namun', 'tahun', 'depan', 'pertumbuhannya', 'berpotensi', 'sedikit', 'menguat', 'hingga', 'tembus', 'persen', 'Dewan', 'Eksekutif', 'IMF', 'Luis', 'E', 'Breuer', 'menyebut', 'perekonomian', 'Indonesia', 'terus', 'menunjukkan', 'perbaikan', 'Hal', 'ini', 'didukung', 'oleh', 'kebijakan', 'makroekonomi', 'yang', 'baik', 'serta', 'reformasi', 'struktural', 'yang', 'terus', 'berlanjut', 'Pemerintah', 'dan', 'otoritas', 'secara', 'terampil', 'mampu', 'melewati', 'tantangan', 'ekonomi', 'terkini', 'yang', 'tengah', 'melanda', 'ekonomi', 'global', 'Pertumbuhan', 'tetap', 'kuat', 'inflasi', 'menurun', 'secara', 'signifikan', 'defisit', 'transaksi', 'berjalan', 'tetap', 'terjaga', 'Kondisi', '

In [9]:
tokens = casefolding(tokens)
print(f"Total token:{len(tokens)}\n")
print(tokens)

Total token:370

['puji' 'ekonomi' 'ri' 'imf' 'ramal' 'pertumbuhan' 'capai' 'persen'
 'ekonomi' 'keuangan' 'suku' 'bunga' 'international' 'monetary' 'fund'
 'imf' 'memproyeksi' 'pertumbuhan' 'ekonomi' 'indonesia' 'hingga' 'akhir'
 'tahun' 'nanti' 'hanya' 'akan' 'mencapai' 'persen' 'namun' 'tahun'
 'depan' 'pertumbuhannya' 'berpotensi' 'sedikit' 'menguat' 'hingga'
 'tembus' 'persen' 'dewan' 'eksekutif' 'imf' 'luis' 'e' 'breuer'
 'menyebut' 'perekonomian' 'indonesia' 'terus' 'menunjukkan' 'perbaikan'
 'hal' 'ini' 'didukung' 'oleh' 'kebijakan' 'makroekonomi' 'yang' 'baik'
 'serta' 'reformasi' 'struktural' 'yang' 'terus' 'berlanjut' 'pemerintah'
 'dan' 'otoritas' 'secara' 'terampil' 'mampu' 'melewati' 'tantangan'
 'ekonomi' 'terkini' 'yang' 'tengah' 'melanda' 'ekonomi' 'global'
 'pertumbuhan' 'tetap' 'kuat' 'inflasi' 'menurun' 'secara' 'signifikan'
 'defisit' 'transaksi' 'berjalan' 'tetap' 'terjaga' 'kondisi' 'ini' 'yang'
 'mampu' 'menopang' 'ekonomi' 'tahun' 'depan' 'ujar' 'breuer' 'dalam

In [10]:
tokens = remove_stopword(tokens)
print(f"Total token:{len(tokens)}\n")
print(tokens)

Total token:235

['puji', 'ekonomi', 'ri', 'imf', 'ramal', 'pertumbuhan', 'capai', 'persen', 'ekonomi', 'keuangan', 'suku', 'bunga', 'international', 'monetary', 'fund', 'imf', 'memproyeksi', 'pertumbuhan', 'ekonomi', 'indonesia', 'mencapai', 'persen', 'pertumbuhannya', 'berpotensi', 'menguat', 'tembus', 'persen', 'dewan', 'eksekutif', 'imf', 'luis', 'e', 'breuer', 'menyebut', 'perekonomian', 'indonesia', 'perbaikan', 'didukung', 'kebijakan', 'makroekonomi', 'reformasi', 'struktural', 'berlanjut', 'pemerintah', 'otoritas', 'terampil', 'melewati', 'tantangan', 'ekonomi', 'terkini', 'melanda', 'ekonomi', 'global', 'pertumbuhan', 'kuat', 'inflasi', 'menurun', 'signifikan', 'defisit', 'transaksi', 'berjalan', 'terjaga', 'kondisi', 'menopang', 'ekonomi', 'breuer', 'keterangan', 'resminya', 'jumat', 'pertbreuer', 'konsumsi', 'rumah', 'tangga', 'indonesia', 'kuat', 'mendorong', 'pertumbuhan', 'ekonomi', 'persen', 'samping', 'konsumsi', 'swasta', 'pertumbuhan', 'investasi', 'swasta', 'dipredik

In [11]:
tokens = stemming(tokens)
print(f"Total token:{len(tokens)}\n")
print(tokens)

Total token:235

['puji', 'ekonomi', 'ri', 'imf', 'ramal', 'tumbuh', 'capai', 'persen', 'ekonomi', 'uang', 'suku', 'bunga', 'international', 'monetary', 'fund', 'imf', 'proyeksi', 'tumbuh', 'ekonomi', 'indonesia', 'capai', 'persen', 'tumbuh', 'potensi', 'kuat', 'tembus', 'persen', 'dewan', 'eksekutif', 'imf', 'luis', 'e', 'breuer', 'sebut', 'ekonomi', 'indonesia', 'baik', 'dukung', 'bijak', 'makroekonomi', 'reformasi', 'struktural', 'lanjut', 'perintah', 'otoritas', 'terampil', 'lewat', 'tantang', 'ekonomi', 'kini', 'landa', 'ekonomi', 'global', 'tumbuh', 'kuat', 'inflasi', 'turun', 'signifikan', 'defisit', 'transaksi', 'jalan', 'jaga', 'kondisi', 'topang', 'ekonomi', 'breuer', 'terang', 'resmi', 'jumat', 'pertbreuer', 'konsumsi', 'rumah', 'tangga', 'indonesia', 'kuat', 'dorong', 'tumbuh', 'ekonomi', 'persen', 'samping', 'konsumsi', 'swasta', 'tumbuh', 'investasi', 'swasta', 'prediksi', 'rangkak', 'dorong', 'bangkit', 'harga', 'komoditas', 'perlahan', 'suku', 'bunga', 'perban', 'rendah

In [12]:
count_tagging(*pos_tagging(tokens))

Verb:40
Noun:141
Adjective:32


In [13]:
most_common_words = nltk.FreqDist(tokens)
print(most_common_words.most_common()[:10])

[('ekonomi', 10), ('persen', 8), ('tumbuh', 7), ('imf', 6), ('uang', 5), ('indonesia', 5), ('kuat', 4), ('perintah', 4), ('jalan', 4), ('bi', 4)]


In [14]:
print(mistaken_stemming(tokens))

['imf', 'international', 'monetary', 'fund', 'imf', 'imf', 'luis', 'breuer', 'breuer', 'pertbreuer']
